# Installation

Session options:
 * Accelerator: GPU T4 x2
 * Language: Python
 * Persistence: File Only
 * Environment: Pin to original environment


In [ ]:
!pip install virtualenv
!python -m venv venv
!/kaggle/working/venv/bin/python -m pip install --upgrade pip # just to be safe
!/kaggle/working/venv/bin/pip install request # example
!/kaggle/working/venv/bin/pip list

In [ ]:
# Set the base image
#ARG BASE_IMAGE
#FROM ${BASE_IMAGE}

# Set the shell and enable pipefail for better error handling
#SHELL ["/bin/bash", "-o", "pipefail", "-c"]

# Set basic environment variables
PYTHON_VERSION=3.12
TORCH_VERSION=2.7.0
CUDA_VERSION=12.6.3
#ENV PREINSTALLED_MODEL

# Set basic environment variables
SHELL=/bin/bash 
PYTHONUNBUFFERED=True 
#DEBIAN_FRONTEND=noninteractive
TZ=America/Sao_Paulo




# Install essential packages (optimized to run in one command)
!apt-get update --yes 
!apt-get upgrade --yes 
!apt-get install --yes --no-install-recommends git wget curl bash nginx-light rsync sudo binutils ffmpeg lshw nano tzdata file build-essential nvtop libgl1 libglib2.0-0 clang libomp-dev ninja-build openssh-server ca-certificates
!apt-get autoremove -y 
!apt-get clean
!rm -rf /var/lib/apt/lists/* /var/cache/apt/archives/*

# Install the UV tool from astral-sh
#ADD https://astral.sh/uv/install.sh /uv-installer.sh
#RUN sh /uv-installer.sh && rm /uv-installer.sh
#ENV PATH="/root/.local/bin/:$PATH"

# Install Python and create virtual environment
#RUN uv python install ${PYTHON_VERSION} --default --preview && \
#    uv venv --seed /venv
#ENV PATH="/workspace/venv/bin:/venv/bin:$PATH"
!pip install uv

# Install essential Python packages and dependencies
RUN pip install --no-cache-dir -U \
    pip setuptools wheel \
    jupyterlab jupyterlab_widgets ipykernel ipywidgets \
    huggingface_hub hf_transfer \
    numpy scipy matplotlib pandas scikit-learn seaborn requests tqdm pillow pyyaml \
    triton \
    torch==${TORCH_VERSION} torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/${CUDA_VERSION}

# Install ComfyUI and ComfyUI Manager
RUN git clone https://github.com/comfyanonymous/ComfyUI.git && \
    cd ComfyUI && \
    pip install --no-cache-dir -r requirements.txt && \
    git clone https://github.com/ltdrdata/ComfyUI-Manager.git custom_nodes/ComfyUI-Manager && \
    cd custom_nodes/ComfyUI-Manager && \
    pip install --no-cache-dir -r requirements.txt

RUN cd /ComfyUI/custom_nodes && \
    git clone --recursive https://github.com/ssitu/ComfyUI_UltimateSDUpscale.git && \
    git clone --recursive https://github.com/receyuki/comfyui-prompt-reader-node.git && \
    git clone https://github.com/comfyanonymous/ComfyUI_TensorRT.git && \
    git clone https://github.com/cubiq/ComfyUI_essentials.git && \
    git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack.git && \
    git clone https://github.com/ltdrdata/ComfyUI-Impact-Subpack.git && \
    git clone https://github.com/jags111/efficiency-nodes-comfyui.git && \
    git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git && \
    git clone https://github.com/JPS-GER/ComfyUI_JPS-Nodes.git && \
    git clone https://github.com/chrisgoringe/cg-use-everywhere.git && \
    git clone https://github.com/crystian/ComfyUI-Crystools.git && \
    git clone https://github.com/rgthree/rgthree-comfy.git && \
    git clone https://github.com/alexopus/ComfyUI-Image-Saver.git && \
    git clone https://github.com/kijai/ComfyUI-WanVideoWrapper.git && \
    git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git && \
    git clone https://github.com/city96/ComfyUI-GGUF.git && \
    git clone https://github.com/kijai/ComfyUI-KJNodes && \
    git clone https://github.com/Flow-two/ComfyUI-WanStartEndFramesNative.git && \
    git clone https://github.com/Smirnov75/ComfyUI-mxToolkit.git && \
    git clone https://github.com/Fannovel16/ComfyUI-Frame-Interpolation.git && \
    find /ComfyUI/custom_nodes -name "requirements.txt" -exec pip install --no-cache-dir -r {} \; && \
    find /ComfyUI/custom_nodes -name "install.py" -exec python {} \;

In [10]:
'''%%time
update = False

import os
import stat
#!rm -rf /kaggle/working/*
#!/usr/local/bin/python -m venv /kaggle/working/venv
home_dir = '/kaggle/working'
python = '/kaggle/working/venv/bin/python'
pip = '/kaggle/working/venv/bin/pip'

def find_bin_folders(folder_path):
    bin_folders = []
    for root, dirs, files in os.walk(folder_path):
        for dir_name in dirs:
            if dir_name == 'bin':
                bin_folders.append(os.path.join(root, dir_name)) 
    return bin_folders

def installLibraries(home_dir, python, pip):
  %cd {home_dir}
  !{pip} install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
  !{pip} install tensorflow[and-cuda]
  #!wget https://q4j3.c11.e2-5.dev/downloads/req.txt
  !{pip} install -r /kaggle/input/requirementforcomfyui/req.txt

!pip install virtualenv

if not os.path.exists(f'{home_dir}/venv'):
    print('installing venv')
    os.chdir(home_dir)
    get_ipython().system(f'cd {home_dir}')
    
    get_ipython().system('virtualenv venv -p $(which python3.10)')
    installLibraries(home_dir, python, pip)
else:
    bin_folders = find_bin_folders('/kaggle/working/venv')
    if bin_folders:
      print("Found 'bin' folders:")
      for bin_folder in bin_folders:
        print(bin_folder)
        for filename in os.listdir(bin_folder):
            file_path = os.path.join(bin_folder, filename)
            if os.path.isfile(file_path):
                current_permissions = os.stat(file_path).st_mode
                 # Add execute permissions for the user, group, and others
                os.chmod(file_path, current_permissions | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)

if not os.path.exists(f'{home_dir}/venv/bin/python3.10'):
    get_ipython().system('cp /usr/bin/python3.10 /kaggle/working/venv/bin/')
!ln -s /kaggle/working/venv/bin/python3.10 /kaggle/working/venv/bin/python
!ln -s /kaggle/working/venv/bin/python3.10 /kaggle/working/venv/bin/python3''''



#!rm -rf /var/lib/apt/lists/* /var/cache/apt/archives/*
'''RUN pip install --no-cache-dir -U \
    pip setuptools wheel \
    jupyterlab jupyterlab_widgets ipykernel ipywidgets \
    huggingface_hub hf_transfer \
    numpy scipy matplotlib pandas scikit-learn seaborn requests tqdm pillow pyyaml \
    triton \
    torch==${TORCH_VERSION} torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/${CUDA_VERSION}'''



SyntaxError: unterminated string literal (detected at line 52) (2856260482.py, line 52)

In [5]:
#!/usr/local/bin/pip install virtualenv
!virtualenv -p python3 /kaggle/working/venv

created virtual environment CPython3.11.13.final.0-64 in 624ms
  creator CPython3Posix(dest=/kaggle/working/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.1.1, setuptools==80.3.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
%%time
#update = False

import os
import stat
!set PYTHONVERBOSE
!apt-get update --yes 
!apt-get upgrade --yes
#--no-install-recommends
!apt-get install --yes lshw libomp-dev ninja-build
!apt-get autoremove -y 
!apt-get clean

!/usr/local/bin/pip install virtualenv
!/usr/local/bin/virtualenv -p python3 /kaggle/working/venv
!source /kaggle/working/venv/bin/activate
#/kaggle/working/venv/bin/pip


!/kaggle/working/venv/bin/pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
#!/kaggle/working/venv/bin/pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu126
!/kaggle/working/venv/bin/pip install tensorflow[and-cuda]
!/kaggle/working/venv/bin/pip install sageattention
#!/kaggle/working/venv/bin/pip install -r /kaggle/input/requirementforcomfyui/req.txt
!/kaggle/working/venv/bin/pip install -r /kaggle/input/requirementsv2/reqv2.txt



%cd /kaggle/working
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd /kaggle/working/ComfyUI
#!git checkout 7fc3ccdcc2fb1f20c4b7dd4aca374db952fd66df
%cd /kaggle/working/ComfyUI
!/kaggle/working/venv/bin/pip install -r /kaggle/working/ComfyUI/requirements.txt

!mkdir /tmp/models
!mkdir /tmp/models/checkpoints
!mkdir /tmp/models/clip
!mkdir /tmp/models/vae
!mkdir /tmp/models/unet

# Remove the following two lines to keep models in permanent storage
!rm -rf /kaggle/working/ComfyUI/models/checkpoints
!rm -rf /kaggle/working/ComfyUI/models/clip
!rm -rf /kaggle/working/ComfyUI/models/vae
!rm -rf /kaggle/working/ComfyUI/models/unet

!ln -sf /tmp/models/checkpoints /kaggle/working/ComfyUI/models/checkpoints
!ln -sf /tmp/models/clip /kaggle/working/ComfyUI/models/clip
!ln -sf /tmp/models/vae /kaggle/working/ComfyUI/models/vae
!ln -sf /tmp/models/unet /kaggle/working/ComfyUI/models/unet

checkpoints =  '/kaggle/working/ComfyUI/models/checkpoints'

#link_path = checkpoints + '/temp-models'
#temp_models = '/kaggle/temp/temp-models'

#!mkdir  /kaggle/temp/
#!mkdir -p $temp_models

#if not os.path.exists(link_path):
#    get_ipython().system(f'ln -s {temp_models} {checkpoints}')

# Install the node manager
#update_manager = True
%cd /kaggle/working/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Manager/
!git pull 
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Manager/requirements.txt

#!git clone https://github.com/Comfy-Org/ComfyUI-Manager.git
#%cd ComfyUI-Manager
#!git pull
%cd /kaggle/working/ComfyUI/custom_nodes
!wget https://gist.githubusercontent.com/city96/30743dfdfe129b331b5676a79c3a8a39/raw/ecb4f6f5202c20ea723186c93da308212ba04cfb/ComfyBootlegOffload.py

#!git clone https://github.com/ZHO-ZHO-ZHO/ComfyUI-Gemini.git
#!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Gemini/requirements.txt
!git clone https://github.com/DoctorDiffusion/ComfyUI-MediaMixer.git
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI-MediaMixer/requirements.txt
!git clone https://github.com/comfyanonymous/ComfyUI_TensorRT.git
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI_TensorRT/requirements.txt
!git clone https://github.com/yuvraj108c/ComfyUI-Upscaler-Tensorrt.git
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Upscaler-Tensorrt/requirements.txt
!git clone https://github.com/city96/ComfyUI-GGUF.git
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI-GGUF/requirements.txt
#!git clone https://github.com/farizrifqi/ComfyUI-Image-Saver.git
#!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Image-Saver/requirements.txt
#!git clone https://github.com/kijai/ComfyUI-MMAudio.git
#!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI-MMAudio/requirements.txt
#!git clone https://github.com/receyuki/comfyui-prompt-reader-node.git
#!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/comfyui-prompt-reader-node/requirements.txt
!git clone https://github.com/ltdrdata/ComfyUI-Impact-Subpack.git
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Impact-Subpack/requirements.txt
!git clone https://github.com/Gourieff/ComfyUI-ReActor.git
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI-ReActor/requirements.txt
!git clone https://github.com/justUmen/Bjornulf_custom_nodes.git
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/Bjornulf_custom_nodes/requirements.txt
#!git clone https://github.com/if-ai/ComfyUI-IF_LLM.git
#!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI-IF_LLM/requirements.txt
!git clone https://github.com/chflame163/ComfyUI_LayerStyle_Advance.git
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI_LayerStyle_Advance/requirements.txt
!git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack.git
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Impact-Pack/requirements.txt
!git clone https://github.com/storyicon/comfyui_segment_anything.git
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/comfyui_segment_anything/requirements.txt
!git clone https://github.com/alt-key-project/comfyui-dream-project.git
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/comfyui-dream-project/requirements.txt
!git clone https://github.com/chflame163/ComfyUI_LayerStyle.git
!/kaggle/working/venv/bin/pip install --no-cache-dir -r /kaggle/working/ComfyUI/custom_nodes/ComfyUI_LayerStyle/requirements.txt

!git clone https://github.com/M1kep/ComfyLiterals.git
!git clone https://github.com/BadCafeCode/masquerade-nodes-comfyui.git
!git clone https://github.com/orssorbit/ComfyUI-wanBlockswap.git
!git clone https://github.com/SeanScripts/ComfyUI-Unload-Model.git
!git clone https://github.com/Smirnov75/ComfyUI-mxToolkit.git
!git clone https://github.com/pharmapsychotic/comfy-cliption.git
!git clone https://github.com/facok/ComfyUI-HunyuanVideoMultiLora.git
!git clone https://github.com/Hullabalo/ComfyUI-Loop.git
!git clone https://github.com/chrisgoringe/cg-use-everywhere.git
!git clone https://github.com/Flow-two/ComfyUI-WanStartEndFramesNative.git
!git clone https://github.com/Chaoses-Ib/ComfyUI_Ib_CustomNodes.git
!git clone https://github.com/JPS-GER/ComfyUI_JPS-Nodes.git
!git clone https://github.com/welltop-cn/ComfyUI-TeaCache.git
!git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git
!git clone https://github.com/kijai/ComfyUI-Florence2.git
!git clone https://github.com/shinich39/comfyui-get-meta.git
!git clone https://github.com/bash-j/mikey_nodes.git
!git clone https://github.com/kijai/ComfyUI-FramePackWrapper.git
!git clone https://github.com/pollockjj/ComfyUI-MultiGPU.git
!git clone https://github.com/cubiq/ComfyUI_essentials.git
!git clone https://github.com/Fannovel16/ComfyUI-Frame-Interpolation.git
!git clone https://github.com/1038lab/ComfyUI-RMBG.git
!git clone https://github.com/ssitu/ComfyUI_UltimateSDUpscale.git
!git clone https://github.com/kijai/ComfyUI-KJNodes.git
!git clone https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes.git
!git clone https://github.com/Fannovel16/comfyui_controlnet_aux.git
!git clone https://github.com/shenduldh/ComfyUI-Lightning.git
!git clone https://github.com/crystian/ComfyUI-Crystools.git
!git clone https://github.com/jags111/efficiency-nodes-comfyui.git
!git clone https://github.com/ltdrdata/ComfyUI-Inspire-Pack.git
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git
!git clone https://github.com/rgthree/rgthree-comfy.git
!git clone https://github.com/yolain/ComfyUI-Easy-Use.git
!git clone https://github.com/ShmuelRonen/ComfyUI-FramePackWrapper_Plus.git
!git clone https://github.com/melMass/comfy_mtb.git
!git clone https://github.com/kijai/ComfyUI-WanVideoWrapper.git
!git clone https://github.com/aria1th/ComfyUI-LogicUtils.git
!git clone https://github.com/MixLabPro/comfyui-mixlab-nodes.git
!git clone https://github.com/WASasquatch/was-node-suite-comfyui.git

#if update_manager:
#    get_ipython().system('git pull')
#!git pull
#!/usr/local/bin/pip install -r /kaggle/working/ComfyUI/custom_nodes/requirements.txt
#!python3 /kaggle/working/ComfyUI/custom_nodes/install.py

# Pinggy script
#!wget https://raw.githubusercontent.com/wandaweb/jupyter-webui-tunneling/main/pinggy.py -O /kaggle/working/pinggy.py
    
# Second GPU offload
#%cd /kaggle/working/ComfyUI/custom_nodes
#!wget https://gist.githubusercontent.com/city96/30743dfdfe129b331b5676a79c3a8a39/raw/ecb4f6f5202c20ea723186c93da308212ba04cfb/ComfyBootlegOffload.py


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,840 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,932 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,119 kB] 
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu jamm

In [9]:
!ssh -o StrictHostKeyChecking=no -i /kaggle/input/tunkey/tun.key -p 8844 -f -T -N -R 8888:localhost:8889 tun@164.152.50.65
#--cache-none  
!python /kaggle/working/ComfyUI/main.py --cache-none --listen 0.0.0.0 --port 8889

[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2025-07-17 22:57:36.435
** Platform: Linux
** Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /kaggle/working/ComfyUI
** ComfyUI Base Folder Path: /kaggle/working/ComfyUI
** User directory: /kaggle/working/ComfyUI/user
** ComfyUI-Manager config path: /kaggle/working/ComfyUI/user/default/ComfyUI-Manager/config.ini
** Log path: /kaggle/working/ComfyUI/user/comfyui.log
[ComfyUI-Manager] 'comfyui-frontend-package' dependency were fixed

Prestartup times for custom nodes:
   0.0 seconds: /kaggle/working/ComfyUI/custom_nodes/rgthree-comfy
   0.0 seconds: /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Easy-Use
  11.7 seconds: /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Manager

Checkpoint files will always be loaded safely.
Total VRAM 15095 MB, total RAM 32103 MB
pytorch version: 2.6.0+cu124
Set vram sta

--- 
# WebUI

## Start the WebUI with Pinggy
* Wait for the GUI to start.  
* Click the link that ends with .pinggy.link 😁
* If generation is still running after the link expires in an hour, wait for the generation to complete and restart the WebUI code block to get a new link

In [ ]:
# Starting the Web UI with pinggy

%cd /kaggle/working/ComfyUI
!python /kaggle/working/pinggy.py --command='/kaggle/working/venv/bin/python /kaggle/working/ComfyUI/main.py' --port=8188

## Start the WebUI with Zrok

### Install Zrok

In [ ]:
# Install Zrok (only needs to run once)

!mkdir /kaggle/working/zrok
%cd /kaggle/working/zrok
!rm zrok*.gz
!wget https://github.com/openziti/zrok/releases/download/v1.0.4/zrok_1.0.4_linux_amd64.tar.gz
!tar -xvf ./zrok*.gz 
!chmod a+x /kaggle/working/zrok/zrok 

### Create a Zrok account
Enter your email address in the email variable

In [ ]:
email = '####@gmail.com' # replace with your email

# --------------

cmd = '/kaggle/working/zrok/zrok invite'
log = '/kaggle/working/zrok/log.txt'

!pip install pexpect
!touch $log

import pexpect
import time
child = pexpect.spawn('bash')
child.sendline(f'{cmd} | tee {log}')
child.expect('enter and confirm your email address...')
time.sleep(1); child.sendline(email); time.sleep(1); child.send(chr(9)); time.sleep(1)
child.sendline(email); time.sleep(1); child.send('\n'); time.sleep(1); child.send(chr(9))
time.sleep(1); child.send('\r\n'); time.sleep(2); child.close()
!cat $log
!rm $log

### Enable Zrok 
Paste your Zrok token in the token variable

In [ ]:
# Enable Zrok (needs to run once per instance)
# Paste your Zrok token in the token variable

token = ""
!chmod a+x /kaggle/working/zrok/zrok 
!/kaggle/working/zrok/zrok enable $token

### Start the WebUI with Zrok

In [ ]:
# Start the WebUI with Zrok
%cd /kaggle/working/ComfyUI
command = '/kaggle/working/venv/bin/python /kaggle/working/ComfyUI/main.py'
port = '8188'
# ------------------------

!chmod a+x /kaggle/working/zrok/zrok 
cmd = f'{command} & /kaggle/working/zrok/zrok share public http://localhost:{port} --headless'
get_ipython().system(cmd)

---
# Model Management

## Install a model

Copy the model URL to the model_url field. Make sure the model can be accessed publicly, without being signed into a website.

In [ ]:
#### Install a model in permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"
model_url = 'https://civitai.com/api/download/models/782002'
model_name = 'JuggernautXL.safetensors'

%cd $checkpoints
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

In [ ]:
# Install a LoRA in permanent storage
model_url = 'https://civitai.com/api/download/models/137124?type=Model&format=SafeTensor'
model_name = 'DreamArt.safetensors'

%cd /kaggle/working/ComfyUI/models/loras
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

In [ ]:
# Install a model in temporary storage
#model_url = 'https://civitai.com/api/download/models/160191?type=Model&format=SafeTensor&size=full&fp=fp16'
#model_name = 'YamersRealism.safetensors'
model_url = 'https://civitai.com/api/download/models/456751'
model_name = 'HelloWorld-XL.safetensors' 

%cd $temp_models
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

## Download a model for a custom node

In [ ]:
model_folder = '/kaggle/working/ComfyUI/custom_nodes/my_node/models'
model_url = ''
model_name = 'model.safetensors'

%cd $model_folder
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

---
# File Browser

## Install FileBrowser

In [ ]:
%cd /kaggle
!wget https://github.com/filebrowser/filebrowser/releases/download/v2.27.0/linux-amd64-filebrowser.tar.gz
!tar xvfz linux-amd64-filebrowser.tar.gz
!chmod a+x /kaggle/filebrowser
!/kaggle/filebrowser config init 
!/kaggle/filebrowser config set --auth.method=noauth > /dev/null
!/kaggle/filebrowser config set --branding.theme=dark > /dev/null
!/kaggle/filebrowser users add admin admin 
!/kaggle/filebrowser config export "/kaggle/config.json"

## Run FileBrowser

In [ ]:
%cd /kaggle
!chmod a+x /kaggle/filebrowser

!python /kaggle/working/pinggy.py --command='/kaggle/filebrowser -c "/kaggle/working/config.json"' --port=8080

# 
# Delete a model

In [ ]:
# List permanent models
!ls -la $checkpoints

# Delete a model
model_to_delete = '/kaggle/working/ComfyUI/models/checkpoints/model.safetensors'
!rm $model_to_delete

In [ ]:
# Check the size of a model
!du -sh /kaggle/working/ComfyUI/models/loras/harrlogos.safetensors

# 
# Delete everything in the working folder

In [ ]:
# Delete the working folder
!rm -rf /kaggle/working/*